In [1]:
import numpy as np
import csv
import pandas as pd
import tensorflow as tf

In [2]:
## 读取csv文件
data_train_origin = []
train = csv.reader(open('train.tsv', 'rt'))
for row in train:
    data_train_origin.append(row)

data_test_origin = []
train = csv.reader(open('test.tsv', 'rt'))
for row in train:
    data_test_origin.append(row)

In [3]:
data_train_origin = data_train_origin[0:1000]
data_test_origin = data_test_origin[0:100]

In [4]:
## 数据的分割
data_train = []
for item in data_train_origin:
    if (len(item)>0):
        for j in range(1,len(item)):
            item[0]+=item[j]
    data_train.append(item[0].split('\t'))

data_test = []
for item in data_test_origin:
    if (len(item)>0):
        for j in range(1,len(item)):
            item[0]+=item[j]
    data_test.append(item[0].split('\t'))
    

In [5]:
## 删除标题
data_train.remove(data_train[0])
data_test.remove(data_test[0])

In [6]:
## 简单预处理
for i in range(len(data_train)):
    data_train[i][0] = int(data_train[i][0])
    data_train[i][1] = int(data_train[i][1])
    data_train[i][2] = data_train[i][2].replace('  ',' ')
    data_train[i][3] = int(data_train[i][3])
    
for i in range(len(data_test)):
    data_test[i][0] = int(data_test[i][0])
    data_test[i][1] = int(data_test[i][1])
    data_test[i][2] = data_test[i][2].replace('  ',' ')

In [7]:
## 切分单词
tokenized_lines = []
for item in data_train:
    tokenized_lines.append(item[2].split(' '))

In [8]:
## 去除标点，替换大小写
punctuation = [',', ':', ';', '.', "'", '"', "’", '?', '/', '-', '+', '&', '(', ')']
clean_tokenized = []
for item in tokenized_lines:
    tokens = []
    for token in item:
        token = token.lower()
        for punc in punctuation:
            token = token.replace(punc,' ')
        tokens.append(token)
    clean_tokenized.append(tokens)

In [9]:
## 读入GloVe所定义的词向量（100维度）
f = open("glove.6B.100d.txt",'r', encoding='UTF-8')  
lines = f.readlines()  
f.close()  

In [10]:
## 对于读入的字典进行预处理，处理成字典的格式
GloVe_dic = {}
for line in lines:
    line = line.replace('\n','')
    line = line.split(" ")
    value = []
    for i in range(1,len(line)):
        value.append(float(line[i]))
    GloVe_dic[line[0]] = value
        

In [11]:
## 计算每个句子的长度并存储
lens = []
for words in clean_tokenized:
    lens.append(len(words))
print(max(lens))

39


In [12]:
## 将每个句子表征称为index的形式，并补全到最大长度
representations_origin = []
for words in clean_tokenized:
    if len(words) < max(lens):
        for i in range(max(lens)-len(words)):
            words.append('_PAD')       
    representations_origin.append(words)     

In [13]:
## 对于每个单词通过字典中的representations来表征
representations = []
for seq in representations_origin:
    representation = []
    for word in seq:
        if (word in GloVe_dic.keys()):
            representation.append(GloVe_dic[word])
        else:
            representation.append(list(np.zeros(100)))
    representations.append(representation)
            
        

OOV通过全0向量来表征

In [14]:
## 定义x，y
x = representations
y = []
for item in data_train:
    y.append(item[3])

In [15]:
## y生成one-hot vector
classes = np.max(y) + 1
y = np.eye(classes)[y]

In [16]:
## 划分集合
n_train = int(len(x)*0.8)
n_test = int(len(x)*0.1)
n_dev = int(len(x) - n_train -n_test)

import random
index_train = random.sample(range(len(x)),n_train)
index_dev = random.sample(list(set(range(len(x)))^set(index_train)) ,n_dev)
index_test = list(set(list(set(range(len(x)))^set(index_train)))^set(index_dev)) 

x_train = []
y_train = []
lens_train = []
for index in index_train:
    x_train.append(x[index])
    y_train.append(y[index])
    lens_train.append(lens[index])

x_dev = []
y_dev = []
lens_dev = []
for index in index_dev:
    x_dev.append(x[index])
    y_dev.append(y[index])
    lens_dev.append(lens[index])

x_test = []
y_test = []
lens_test = []
for index in index_test:
    x_test.append(x[index])
    y_test.append(y[index])
    lens_test.append(lens[index])

x_train = np.array(x_train)
y_train = np.array(y_train)

x_dev = np.array(x_dev)
y_dev = np.array(y_dev)

x_test = np.array(x_test)
y_test = np.array(y_test)

In [17]:
## 定义一个随机函数，随机抽取
import random
def random_sample(x,y,lens_):
    index = random.sample(range(len(x)),1)[0]
    balanced_m = np.concatenate((np.ones((lens_[index], 5),dtype = np.float32),np.zeros((max(lens)-lens_[index], 5),dtype = np.float32)))
    return x[index],y[index],balanced_m

In [18]:
## 通过tf构建logistic model
# 定义输入和输出,以及一个平衡矩阵
x_input = tf.placeholder(tf.float32,[max(lens),100])
y_label = tf.placeholder(tf.float32,[5])
balanced_matrix = tf.placeholder(tf.float32,[max(lens),5])

In [19]:
# 定义参数
W = tf.Variable(tf.random_normal([100,5],stddev=1))
b = tf.Variable(tf.random_normal([1,5],stddev=1))

In [20]:
# 计算预测值y_,通过对每个词的预测求平均获得对于整个文本的情感分析
y_pre = tf.reduce_mean(tf.nn.sigmoid(tf.matmul(x_input,W)+b)*balanced_matrix,axis=0)

In [21]:
# 计算loss function
L = -tf.reduce_mean(y_label*tf.log(y_pre) + (1-y_label)*tf.log(1-y_pre))

In [22]:
# 梯度下降法
train_step = tf.train.GradientDescentOptimizer(0.000001).minimize(L)

In [23]:
# 初始化variables
init = tf.global_variables_initializer()

In [36]:
# 进行训练
L_train = []
L_dev = []
n = 0
m = 0

with tf.Session() as sess:
    sess.run(init)
    while (n < 20 and m< 100):
        x_data,y_data,balanced_m = random_sample(x_train,y_train,lens_train)      
        x_dev_data,y_dev_data,balanced_dev_m = random_sample(x_dev,y_dev,lens_dev)

        sess.run(train_step,feed_dict={x_input:x_data,y_label:y_data,balanced_matrix:balanced_m})
        L_dev.append(sess.run(L,feed_dict={x_input:x_dev_data,y_label:y_dev_data,balanced_matrix:balanced_dev_m}))
        L_train.append(sess.run(L,feed_dict={x_input:x_data,y_label:y_data,balanced_matrix:balanced_m}))
        m += 1
        if (len(L_dev)>=2):
            if (L_dev[-1] > L_dev[-2]):
                n += 1
            else:
                n = 0
        print (str(m)+' iteration: ',n)
        
    W_ = sess.run(W)
    b_ = sess.run(b)
    y_test_pre = []
    for j in range(len(x_test)):
        balanced_test_m = np.concatenate((np.ones((lens_test[j], 5),dtype = np.float32),np.zeros((max(lens)-lens_test[j], 5),dtype = np.float32)))
        
        y_test_pre.append(sess.run(y_pre,feed_dict={x_input:x_test[j],y_label:y_test[j],balanced_matrix:balanced_test_m}))
        

1 iteration:  0
2 iteration:  0
3 iteration:  0
4 iteration:  1
5 iteration:  2
6 iteration:  0
7 iteration:  1
8 iteration:  0
9 iteration:  1
10 iteration:  2
11 iteration:  0
12 iteration:  1
13 iteration:  2
14 iteration:  3
15 iteration:  0
16 iteration:  1
17 iteration:  0
18 iteration:  0
19 iteration:  1
20 iteration:  0
21 iteration:  0
22 iteration:  1
23 iteration:  2
24 iteration:  0
25 iteration:  1
26 iteration:  0
27 iteration:  0
28 iteration:  1
29 iteration:  0
30 iteration:  0
31 iteration:  1
32 iteration:  0
33 iteration:  1
34 iteration:  0
35 iteration:  1
36 iteration:  0
37 iteration:  1
38 iteration:  0
39 iteration:  0
40 iteration:  0
41 iteration:  1
42 iteration:  2
43 iteration:  0
44 iteration:  0
45 iteration:  0
46 iteration:  1
47 iteration:  0
48 iteration:  1
49 iteration:  0
50 iteration:  0
51 iteration:  1
52 iteration:  0
53 iteration:  1
54 iteration:  2
55 iteration:  0
56 iteration:  0
57 iteration:  1
58 iteration:  2
59 iteration:  0
60 ite

In [71]:
## 设计模型在test set上的验证
def evaluate(W,b,x_test,y_test):
    y = []
    for onehot in y_test:
        y.append(np.where(onehot > 0.5)[0][0])
    y_pre  = np.average(1/(1+np.exp(-np.dot(x_test,W)-b)),axis=1)
    y_ = []
    for vector in y_pre:
        y_.append(np.where(vector == max(vector))[0][0])
    loss = -np.average(y_test*np.log(y_pre)+(1-y_test)*np.log(1-y_pre))
    accuracy = float(np.sum(np.array(y)==np.array(y_)))/float(len(y))
    precision = []
    recall = []
    f1 = []
    elems = list(set(y))
    for elem in elems:
        p = 0
        r = 0
        n = 0
        for i in range(len(y)):
            if (y[i] == elem):
                p += 1
            if (y_[i] == elem):
                r += 1
            if (y[i] == y_[i] and y[i] == elem):
                n +=1
        if (p != 0):
            precision.append(float(n)/float(p))
        else:
            precision.append(0)
        if (r != 0):
            recall.append(float(n)/float(r))
        else:
            recall.append(0)
    for j in range(len(precision)):
        if (precision[j]!=0 or recall[j] != 0):
            f1.append(2*precision[j]*recall[j]/(precision[j]+recall[j]))
        else:
            f1.append(0)
    return loss,accuracy,elems,precision,recall,f1

In [72]:
# evaluate
loss,accuracy,elems,precision,recall,f1 = evaluate(W_,b_,x_test,y_test)

In [64]:
loss

0.5717874967868759

In [65]:
accuracy

0.6363636363636364

In [66]:
precision

[0.0, 0.0, 1.0, 0.0, 0.0]

In [67]:
recall

[0.0, 0, 0.6428571428571429, 0, 0]

In [68]:
f1

[0, 0, 0.782608695652174, 0, 0]